# Anatomy of a QualibrationGraph

This guide dissects a script that defines and runs a Qualibration Graph, using `91_calibration_graph_retuning_fixed_frequency_transmon.py` as a concrete example. Calibration graphs allow you to chain multiple calibration nodes together to perform complex, automated calibration sequences. We will go through the script section by section, presenting the code first, followed by a detailed explanation of its purpose, focusing on how graphs are constructed and executed programmatically.

## Imports

In [ ]:
from typing import List
from qualibrate.orchestration.basic_orchestrator import BasicOrchestrator
from qualibrate.parameters import GraphParameters
from qualibrate.qualibration_graph import QualibrationGraph
from qualibrate.qualibration_library import QualibrationLibrary

### Explanation: Imports Section

This section imports the necessary classes for defining and running a calibration graph:
- `List` from `typing`: Used for type hinting lists (e.g., list of qubit names).
- `BasicOrchestrator` from `qualibrate.orchestration.basic_orchestrator`: A standard orchestrator class that manages the execution flow of the graph (e.g., running nodes sequentially based on connectivity).
- `GraphParameters` from `qualibrate.parameters`: A base class for defining parameters that apply to the entire graph execution (like the list of target qubits).
- `QualibrationGraph` from `qualibrate.qualibration_graph`: The main class used to define the structure and components of a calibration graph.
- `QualibrationLibrary` from `qualibrate.qualibration_library`: Used to access the library of available calibration nodes that have been discovered by QUAlibrate.

## Load Library

In [ ]:
# Get the active library containing discovered calibration nodes
library = QualibrationLibrary.get_active_library()

### Explanation: Load Library Section

- `library = QualibrationLibrary.get_active_library()`: Retrieves the collection of all calibration nodes that QUAlibrate has discovered (typically from the `calibration_graph` folder). This `library` object acts as a repository from which nodes can be selected and copied to build the graph.

## Define Graph Parameters

In [ ]:
# Define parameters specific to this graph execution
class Parameters(GraphParameters):
    qubits: List[str] = ["q1"]  # Default target qubit(s)

### Explanation: Define Graph Parameters Section

- `class Parameters(GraphParameters): ...`: Defines a class inheriting from `GraphParameters`. This holds parameters that apply globally to the graph execution, such as the list of target `qubits` for which the graph sequence should be run. Default values can be provided here, which can be overridden when the graph is executed.

## Instantiate Graph

In [ ]:
g = QualibrationGraph(
    name="FixedFrequencyTransmon_Retuning",
    parameters=Parameters(),
    nodes={
        "IQ_blobs": library.nodes["07_iq_blobs"].copy(name="IQ_blobs"),
        "ramsey": library.nodes["06a_ramsey"].copy(name="ramsey", use_state_discrimination=True),
        "power_rabi_error_amplification_x180": library.nodes["04b_power_rabi"].copy(
            name="power_rabi_error_amplification_x180",
            max_number_pulses_per_sweep=200,
            min_amp_factor=0.98,
            max_amp_factor=1.02,
            amp_factor_step=0.002,
            use_state_discrimination=True,
        ),
        "power_rabi_error_amplification_x90": library.nodes["04b_power_rabi"].copy(
            name="power_rabi_error_amplification_x90",
            max_number_pulses_per_sweep=200,
            min_amp_factor=0.98,
            max_amp_factor=1.02,
            amp_factor_step=0.002,
            operation="x90",
            update_x90=False,
            use_state_discrimination=True,
        ),
        "Randomized_benchmarking": library.nodes["11a_single_qubit_randomized_benchmarking"].copy(
            name="Randomized_benchmarking",
            use_state_discrimination=True,
            delta_clifford=20,
            num_random_sequences=500,
        ),
    },
    connectivity=[
        ("IQ_blobs", "ramsey"),
        ("ramsey", "power_rabi_error_amplification_x180"),
        ("power_rabi_error_amplification_x180", "power_rabi_error_amplification_x90"),
        ("power_rabi_error_amplification_x90", "Randomized_benchmarking"),
    ],
    orchestrator=BasicOrchestrator(skip_failed=False),
)

### Explanation: Instantiate Graph Section

- `g = QualibrationGraph(...)`: Creates the graph object. Key arguments are:
    - `name`: A unique name for this specific graph definition (e.g., "FixedFrequencyTransmon_Retuning").
    - `parameters`: An instance of the `Parameters` class defined above, linking the graph-level parameters.
    - `nodes`: A dictionary defining the nodes included in this graph.
        - Each key is a unique identifier for the node *within this graph* (e.g., "IQ_blobs", "ramsey"). These names are used in the `connectivity` list.
        - Each value is created by copying a node from the `library` using `library.nodes["<original_node_name>"].copy(...)`. The `.copy()` method is essential to ensure that modifications made here (like renaming or overriding parameters) don't affect the original node definition in the library.
        - Node instances within the graph can be renamed using the `name` argument in `.copy()`. If omitted, the graph-internal name (the dictionary key) is used.
        - Default parameters of a node can be overridden for its specific instance within this graph by passing them as keyword arguments to `.copy()` (e.g., `use_state_discrimination=True` is passed to the "ramsey" node copy).
    - `connectivity`: A list of tuples defining the execution dependencies. Each tuple `(source_node_name, target_node_name)` indicates that the node identified by `target_node_name` (the key in the `nodes` dictionary) should run after the node identified by `source_node_name` completes successfully (behavior depends on the orchestrator). This defines the execution flow of the graph. Nodes not listed as targets are starting points.
    - `orchestrator`: An instance of an orchestrator class that manages how the graph is executed. `BasicOrchestrator` runs nodes sequentially based on the `connectivity` list. `skip_failed=False` means the graph execution will stop if any node fails; setting it to `True` would allow the graph to continue with other independent branches if one fails.

## Run Graph

In [ ]:
# Run the graph, passing runtime parameters (overriding defaults if needed)
# Example: Run for multiple qubits
# g.run(qubits=[f"q{i+1}" for i in range(0, 4)])
# Example: Run for specific qubits
g.run(qubits=["q1", "q3"])

### Explanation: Run Graph Section

- `g.run(qubits=["q1", "q3"])`: Executes the defined graph `g`.
- Runtime parameters (like the specific `qubits` to run the calibration sequence on) are passed as keyword arguments to the `.run()` method. These values override the defaults defined in the graph's `Parameters` class for this specific execution.
- The orchestrator manages the execution, iterating through the specified qubits (or other parallelizable parameters defined in the orchestrator or graph parameters) and running the sequence of connected nodes for each one according to the defined `connectivity` and orchestrator logic.